In [4]:
import gym

class GymRunner():

    env = None

    def __init__(env_name = "LunarLander-v2"):
        self.env = gym.make(env_name)
        #self.env._max_episode_steps = 1200

    def get_shapes():
        return {
            "action_space" : self.env.action_space.n,
            "observation_space" : self.env.observation_space.shape,
        }

    def test_agent(agent, n_iters):
        done = False
        observation = self.env.reset()
        reward = 0
        info = {}

        for _ in range(n_iters):
            while not done:
                self.env.render()

                action = agent.act(observation, reward, done)

                observation, reward, done, info = self.env.step(action)

                print(f"REWARD = {reward}")

In [7]:
import argparse
import sys

import gym
from gym import wrappers, logger

class RandomAgent(object):
    """The world's simplest agent!"""
    def __init__(self, action_space):
        self.action_space = action_space

    def act(self, observation, reward, done):
        return self.action_space.sample()

# New comment
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description=None)
    parser.add_argument('env_id', nargs='?', default='LunarLander-v2', help='Select the environment to run')
    args = parser.parse_args()

    # You can set the level to logger.DEBUG or logger.WARN if you
    # want to change the amount of output.
    logger.set_level(logger.INFO)
    
    env = gym.make(args.env_id)

    # You provide the directory to write to (can be an existing
    # directory, including one with existing data -- all monitor files
    # will be namespaced). You can also dump to a tempdir if you'd
    # like: tempfile.mkdtemp().
    outdir = '/tmp/random-agent-results'
    env = wrappers.Monitor(env, directory=outdir, force=True)
    env.seed(0)
    agent = RandomAgent(env.action_space)

    episode_count = 100
    reward = 0
    done = False

    for i in range(episode_count):
        ob = env.reset()
        while True:
            action = agent.act(ob, reward, done)
            ob, reward, done, _ = env.step(action)
            if done:
                break
            # Note there's no env.render() here. But the environment still can open window and
            # render if asked by env.monitor: it calls env.render('rgb_array') to record video.
            # Video is not recorded every episode, see capped_cubic_video_schedule for details.

    # Close the env and write monitor result info to disk
    env.close()

usage: ipykernel_launcher.py [-h] [env_id]
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

In [3]:
import gym
env = gym.make('LunarLander-v2')
env.render()
env.reset()
print("hello")

hello


In [5]:
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.cem import CEMAgent
from rl.memory import EpisodeParameterMemory

ENV_NAME = 'CartPole-v1'


# Get the environment and extract the number of actions.
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)

nb_actions = env.action_space.n
obs_dim = env.observation_space.shape[0]

# Option 1 : Simple model
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(nb_actions))
model.add(Activation('softmax'))

# Option 2: deep network
# model = Sequential()
# model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
# model.add(Dense(16))
# model.add(Activation('relu'))
# model.add(Dense(16))
# model.add(Activation('relu'))
# model.add(Dense(16))
# model.add(Activation('relu'))
# model.add(Dense(nb_actions))
# model.add(Activation('softmax'))


print(model.summary())


# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = EpisodeParameterMemory(limit=1000, window_length=1)

cem = CEMAgent(model=model, nb_actions=nb_actions, memory=memory,
               batch_size=50, nb_steps_warmup=2000, train_interval=50, elite_frac=0.05)
cem.compile()

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
cem.fit(env, nb_steps=100000, visualize=False, verbose=2)

# After training is done, we save the best weights.

#cem.save_weights('cem_{}_params.h5f'.format(ENV_NAME), overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
cem.test(env, nb_episodes=5, visualize=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 10        
_________________________________________________________________
activation_2 (Activation)    (None, 2)                 0         
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
None
Training for 100000 steps ...
    13/100000: episode: 1, duration: 0.039s, episode steps: 13, steps per second: 334, episode reward: 13.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.538 [0.000, 1.000], mean observation: -0.107 [-1.319, 0.772], mean_best_reward: --
    24/100000: episode: 2, duration: 0.005s, episode steps: 11, steps per second: 2243, episode reward: 11.000, mean reward: 1.000 

   893/100000: episode: 44, duration: 0.004s, episode steps: 10, steps per second: 2493, episode reward: 10.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.200 [0.000, 1.000], mean observation: 0.145 [-1.523, 2.391], mean_best_reward: --
   924/100000: episode: 45, duration: 0.010s, episode steps: 31, steps per second: 3144, episode reward: 31.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.613 [0.000, 1.000], mean observation: -0.034 [-2.329, 1.418], mean_best_reward: --
   939/100000: episode: 46, duration: 0.009s, episode steps: 15, steps per second: 1640, episode reward: 15.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.733 [0.000, 1.000], mean observation: -0.066 [-2.251, 1.410], mean_best_reward: --
   955/100000: episode: 47, duration: 0.006s, episode steps: 16, steps per second: 2595, episode reward: 16.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.750 [0.000, 1.000], mean observation: -0.060 [-2.520, 1.606], mean_best_reward: --
   998/100000: ep

  1720/100000: episode: 79, duration: 0.011s, episode steps: 13, steps per second: 1168, episode reward: 13.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.692 [0.000, 1.000], mean observation: -0.094 [-1.725, 1.021], mean_best_reward: --
  1729/100000: episode: 80, duration: 0.007s, episode steps: 9, steps per second: 1353, episode reward: 9.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.889 [0.000, 1.000], mean observation: -0.136 [-2.446, 1.607], mean_best_reward: --
  1741/100000: episode: 81, duration: 0.006s, episode steps: 12, steps per second: 1886, episode reward: 12.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.250 [0.000, 1.000], mean observation: 0.117 [-1.355, 2.126], mean_best_reward: --
  1752/100000: episode: 82, duration: 0.008s, episode steps: 11, steps per second: 1456, episode reward: 11.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.818 [0.000, 1.000], mean observation: -0.125 [-2.329, 1.418], mean_best_reward: --
  1771/100000: epis

  2829/100000: episode: 121, duration: 0.027s, episode steps: 88, steps per second: 3234, episode reward: 88.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.500 [0.000, 1.000], mean observation: 0.072 [-0.497, 1.342], mean_best_reward: --
  2973/100000: episode: 122, duration: 0.053s, episode steps: 144, steps per second: 2728, episode reward: 144.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.493 [0.000, 1.000], mean observation: 0.116 [-1.161, 1.565], mean_best_reward: --
  2990/100000: episode: 123, duration: 0.007s, episode steps: 17, steps per second: 2428, episode reward: 17.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.588 [0.000, 1.000], mean observation: -0.119 [-1.588, 0.759], mean_best_reward: --
  3019/100000: episode: 124, duration: 0.010s, episode steps: 29, steps per second: 2870, episode reward: 29.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.517 [0.000, 1.000], mean observation: 0.126 [-0.499, 1.035], mean_best_reward: --
  3046/100000

  4022/100000: episode: 157, duration: 0.008s, episode steps: 22, steps per second: 2638, episode reward: 22.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.500 [0.000, 1.000], mean observation: 0.100 [-0.415, 1.031], mean_best_reward: --
  4064/100000: episode: 158, duration: 0.015s, episode steps: 42, steps per second: 2812, episode reward: 42.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.571 [0.000, 1.000], mean observation: 0.054 [-1.553, 1.533], mean_best_reward: --
  4126/100000: episode: 159, duration: 0.022s, episode steps: 62, steps per second: 2834, episode reward: 62.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.468 [0.000, 1.000], mean observation: 0.021 [-1.123, 1.374], mean_best_reward: --
  4147/100000: episode: 160, duration: 0.007s, episode steps: 21, steps per second: 2847, episode reward: 21.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.381 [0.000, 1.000], mean observation: 0.052 [-1.119, 1.759], mean_best_reward: --
  4199/100000: e

...

 92209/100000: episode: 2324, duration: 0.016s, episode steps: 46, steps per second: 2839, episode reward: 46.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.522 [0.000, 1.000], mean observation: 0.100 [-0.587, 1.398], mean_best_reward: --
 92229/100000: episode: 2325, duration: 0.012s, episode steps: 20, steps per second: 1670, episode reward: 20.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.550 [0.000, 1.000], mean observation: -0.060 [-1.352, 0.830], mean_best_reward: --
 92282/100000: episode: 2326, duration: 0.023s, episode steps: 53, steps per second: 2330, episode reward: 53.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.491 [0.000, 1.000], mean observation: 0.174 [-0.976, 1.638], mean_best_reward: --
 92409/100000: episode: 2327, duration: 0.050s, episode steps: 127, steps per second: 2556, episode reward: 127.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.520 [0.000, 1.000], mean observation: 0.338 [-0.968, 1.851], mean_best_reward: --
 92429/10

 94000/100000: episode: 2362, duration: 0.014s, episode steps: 46, steps per second: 3195, episode reward: 46.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.457 [0.000, 1.000], mean observation: 0.061 [-1.000, 1.732], mean_best_reward: --
 94075/100000: episode: 2363, duration: 0.029s, episode steps: 75, steps per second: 2583, episode reward: 75.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.467 [0.000, 1.000], mean observation: -0.167 [-1.280, 1.067], mean_best_reward: --
 94099/100000: episode: 2364, duration: 0.008s, episode steps: 24, steps per second: 3056, episode reward: 24.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.417 [0.000, 1.000], mean observation: 0.083 [-0.936, 1.674], mean_best_reward: --
 94137/100000: episode: 2365, duration: 0.012s, episode steps: 38, steps per second: 3276, episode reward: 38.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.447 [0.000, 1.000], mean observation: 0.101 [-0.766, 1.814], mean_best_reward: --
 94151/1000

 95744/100000: episode: 2409, duration: 0.021s, episode steps: 70, steps per second: 3337, episode reward: 70.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.486 [0.000, 1.000], mean observation: -0.136 [-1.291, 0.943], mean_best_reward: --
 95846/100000: episode: 2410, duration: 0.040s, episode steps: 102, steps per second: 2521, episode reward: 102.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.480 [0.000, 1.000], mean observation: -0.250 [-1.535, 1.139], mean_best_reward: --
 95904/100000: episode: 2411, duration: 0.018s, episode steps: 58, steps per second: 3250, episode reward: 58.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.466 [0.000, 1.000], mean observation: -0.064 [-1.372, 1.337], mean_best_reward: --
 95948/100000: episode: 2412, duration: 0.013s, episode steps: 44, steps per second: 3267, episode reward: 44.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.455 [0.000, 1.000], mean observation: 0.003 [-1.025, 1.580], mean_best_reward: --
 95971/

 97695/100000: episode: 2452, duration: 0.010s, episode steps: 25, steps per second: 2492, episode reward: 25.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.600 [0.000, 1.000], mean observation: -0.078 [-2.069, 1.144], mean_best_reward: --
 97706/100000: episode: 2453, duration: 0.005s, episode steps: 11, steps per second: 2226, episode reward: 11.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.636 [0.000, 1.000], mean observation: -0.094 [-1.588, 1.012], mean_best_reward: --
 97721/100000: episode: 2454, duration: 0.007s, episode steps: 15, steps per second: 2262, episode reward: 15.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.600 [0.000, 1.000], mean observation: -0.048 [-1.696, 1.218], mean_best_reward: --
 97746/100000: episode: 2455, duration: 0.009s, episode steps: 25, steps per second: 2696, episode reward: 25.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.480 [0.000, 1.000], mean observation: -0.062 [-0.987, 0.606], mean_best_reward: --
 97774/1

 99483/100000: episode: 2497, duration: 0.019s, episode steps: 52, steps per second: 2711, episode reward: 52.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.500 [0.000, 1.000], mean observation: -0.106 [-1.339, 0.539], mean_best_reward: --
 99501/100000: episode: 2498, duration: 0.008s, episode steps: 18, steps per second: 2241, episode reward: 18.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.444 [0.000, 1.000], mean observation: 0.082 [-1.141, 1.790], mean_best_reward: --
 99585/100000: episode: 2499, duration: 0.026s, episode steps: 84, steps per second: 3206, episode reward: 84.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.512 [0.000, 1.000], mean observation: 0.076 [-1.255, 1.031], mean_best_reward: --
 99606/100000: episode: 2500, duration: 0.008s, episode steps: 21, steps per second: 2649, episode reward: 21.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.619 [0.000, 1.000], mean observation: -0.083 [-2.023, 1.017], mean_best_reward: --
 99703/100

In [42]:
env.action_space.np_random

In [12]:
import rl

In [13]:
rl.agents

In [4]:
import gym
env = gym.make('CartPole-v1')
mn = []
for i_episode in range(10):
    observation = env.reset()
    for t in range(100):
        env.render()
        
        
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        
        
        if done:
            print("Episode finished after {} timesteps".format(t+1),', Reward:',reward)
            break

Episode finished after 22 timesteps , Reward: 1.0
Episode finished after 14 timesteps , Reward: 1.0
Episode finished after 23 timesteps , Reward: 1.0
Episode finished after 12 timesteps , Reward: 1.0
Episode finished after 51 timesteps , Reward: 1.0
Episode finished after 14 timesteps , Reward: 1.0
Episode finished after 13 timesteps , Reward: 1.0
Episode finished after 10 timesteps , Reward: 1.0
Episode finished after 49 timesteps , Reward: 1.0
Episode finished after 49 timesteps , Reward: 1.0
